In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import json
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


# 1. Convert JSON data to a Pandas DataFrame and explore the structure, time period, and scope of the data.

# Checkin

In [ ]:
file_checkin = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json")
checkin = []
for line in file_checkin:
    checkin.append(json.loads(line))

records = []
for entry in checkin:
    business_id = entry['business_id']
    date = entry['date'].split(", ")

    for time in date:
        records.append({
            'business_id': business_id,
            'date': time
        })
checkin_df = pd.DataFrame(records)
print(checkin_df.head())
#checkin_df.to_csv("checkin_data.csv",index=False)
file_checkin.close()

              business_id                 date
0  ---kPU91CF4Lq2-WlRu9Lw  2020-03-13 21:10:56
1  ---kPU91CF4Lq2-WlRu9Lw  2020-06-02 22:18:06
2  ---kPU91CF4Lq2-WlRu9Lw  2020-07-24 22:42:27
3  ---kPU91CF4Lq2-WlRu9Lw  2020-10-24 21:36:13
4  ---kPU91CF4Lq2-WlRu9Lw  2020-12-09 21:23:33


# Business

In [ ]:
file_business = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json")
business = []
for line in file_business:
    business.append(json.loads(line))

business_df = pd.json_normalize(business)

business_df['categories'] = business_df['categories'].str.split(", ")

#business_df.to_excel("business_data.xlsx", index=False)
print(business_df.head())
file_business.close()

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  ...  \
0  34.426679 -119.711197    5.0             7  ...   
1  38.551126  -90.335695    3.0            15  ...   
2  32.223236 -110.880452    3.5            22  ...   
3  39.955505  -75.155564  

# Riview

In [ ]:
file_review = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json")

chunk_size = 10000
review_chunks = []
review_df = pd.DataFrame()

for i, line in enumerate(file_review):
    try:
        review_data = json.loads(line)
        review_chunks.append(review_data)

        if (i + 1) % chunk_size == 0:
            chunk_df = pd.json_normalize(review_chunks)
            review_chunks = []

            review_df = pd.concat([review_df, chunk_df], ignore_index=True)

    except json.JSONDecodeError as e:
        print(f"JSON decode error at line {i}: {e}")

if review_chunks:
    chunk_df = pd.json_normalize(review_chunks)
    review_df = pd.concat([review_df, chunk_df], ignore_index=True)

print(review_df.head())
#review_df.to_csv("review_data.csv",index=False)
file_review.close()

                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0    3.0       0      0     0   
1    5.0       1      0     1   
2    3.0       0      0     0   
3    5.0       1      0     1   
4    4.0       1      0     1   

                                                text                 date  
0  If you decide to eat here, just be aware it is...  2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year...  2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm...  2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delici

# Tip

In [ ]:
file_tip = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json")
tip = []
for line in file_tip:
    tip.append(json.loads(line))

tip_df = pd.json_normalize(tip)
print(tip_df.head())
#tip_df.to_csv("tip_data.csv",index=False)
file_tip.close()

                  user_id             business_id  \
0  AGNUgVwnZUey3gcPCJ76iw  3uLgwr0qeCNMjKenHJwPGQ   
1  NBN4MgHP9D3cw--SnauTkA  QoezRbYQncpRqyrLH6Iqjg   
2  -copOvldyKh1qr-vzkDEvw  MYoRNLb5chwjQe3c_k37Gg   
3  FjMQVZjSqY8syIO-53KFKw  hV-bABTK-glh5wj31ps_Jw   
4  ld0AperBXk1h6UbqmM80zw  _uN0OudeJ3Zl_tf6nxg5ww   

                                                text                 date  \
0                     Avengers time with the ladies.  2012-05-18 02:17:21   
1  They have lots of good deserts and tasty cuban...  2013-02-05 18:35:10   
2             It's open even when you think it isn't  2013-08-18 00:56:08   
3                          Very decent fried chicken  2017-06-27 23:05:38   
4             Appetizers.. platter special for lunch  2012-10-06 19:43:09   

   compliment_count  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  


# User

In [ ]:
file_user = open("/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json")

chunk_size = 10000
user_chunks = []
user_df = pd.DataFrame()

for i, line in enumerate(file_user):
    try:
        user_data = json.loads(line)
        user_chunks.append(user_data)

        if (i + 1) % chunk_size == 0:
            chunk_df = pd.json_normalize(user_chunks)
            user_chunks = []

            user_df = pd.concat([user_df, chunk_df], ignore_index=True)

    except json.JSONDecodeError as e:
        print(f"JSON decode error at line {i}: {e}")

if user_chunks:
    chunk_df = pd.json_normalize(user_chunks)
    user_df = pd.concat([user_df, chunk_df], ignore_index=True)

print(user_df.head())
# user_df['friends'] = user_df['friends'].str.split(", ")
# user_df['elite'] = user_df['elite'].str.split(", ")
# user_df.to_csv("user_data.csv", index=False)
file_user.close()

                  user_id    name  review_count        yelping_since  useful  \
0  qVc8ODYU5SZjKXVBgXdI7w  Walker           585  2007-01-25 16:47:26    7217   
1  j14WgRoU_-2ZE1aw1dXrJg  Daniel          4333  2009-01-25 04:35:42   43091   
2  2WnXYQFK0hXEoTxPtV2zvg   Steph           665  2008-07-25 10:41:00    2086   
3  SZDeASXq7o05mMNLshsdIA    Gwen           224  2005-11-29 04:38:33     512   
4  hA5lMy-EnncsH4JoR-hFGQ   Karen            79  2007-01-05 19:40:59      29   

   funny   cool                                              elite  \
0   1259   5994                                               2007   
1  13066  27281  2009,2010,2011,2012,2013,2014,2015,2016,2017,2...   
2   1010   1003                           2009,2010,2011,2012,2013   
3    330    299                                     2009,2010,2011   
4     15      7                                                      

                                             friends  fans  ...  \
0  NSCy54eWehBJyZdG2iE84w, pe42

# 2. Set up the environment and connect to the database using the IP address and adjust the dataframe to upload it to the Azure database

In [ ]:
from sqlalchemy import create_engine
import sqlalchemy
!pip install pyodbc
!pip install sqlalchemy
!apt-get install -y unixodbc-dev

!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install -y msodbcsql17

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unixodbc-dev is already the newest version (2.3.11-1).
0 upgraded, 0 newly installed, 0 to remove and 58 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  10215      0 --:--:-- --:--:-- --:--:-- 10239
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    89  100    89    0     0    955      0 --:--:-- --:--:-- --:--:--   956
Hit:1 https://packages.cloud.google.com/apt gcsfuse-focal InRelease
Hit:2 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease              
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease    
Hit:5 http://archive.ubuntu.com

In [ ]:
from sqlalchemy import create_engine
import sqlalchemy
!pip install pyodbc
!pip install sqlalchemy

# Define connection details
username = 'sqladmin'  # Replace with your Azure SQL admin username
password = 'MINIproject11'        # Replace with your Azure SQL password
server = 'zienzhu2-server.database.windows.net'
database = 'mini-project-db'

# Connection string for Azure SQL Server
connection_string = f"mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(connection_string)

In [ ]:
import requests
# GET IP
ip_address = requests.get('https://api64.ipify.org').text
print("Your IP address is:", ip_address)

Your IP address is: 35.230.36.89


In [ ]:
tip_df.to_sql(name="tip_df", con=engine, if_exists='replace', index=False, chunksize=5000)

70915

In [ ]:
review_df.to_sql(name="review_df", con=engine, if_exists='replace', index=False, chunksize=5000)

149633

In [ ]:
user_df.to_sql(name="user_df", con=engine, if_exists='replace', index=False, chunksize=5000)

In [ ]:
# Because the business table is special, we need to adjust its JSON format,
# otherwise one of its columns called categories will become null.

import pandas as pd
import json
from sqlalchemy import create_engine
import sqlalchemy

# Convert data of dict type to JSON string format
business_df['attributes'] = business_df['attributes'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)
business_df['hours'] = business_df['hours'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

# Upload to the database, specifying data types
business_df.to_sql(
    name="business_df",
    con=engine,
    if_exists='replace',
    index=True,
    dtype={
        'attributes': sqlalchemy.types.NVARCHAR,
        'hours': sqlalchemy.types.NVARCHAR
    },
    chunksize=5000
)

66

In [ ]:
checkin_df.to_sql(name="checkin_df", con=engine, if_exists='replace', index=False, chunksize=5000)